# Mutagenesis Example
Following example demonstrates learning to [predict the mutagenicity on Salmonella typhimurium](https://relational.fit.cvut.cz/dataset/Mutagenesis) (dataset is stored in json format [in MLDatasets.jl](https://juliaml.github.io/MLDatasets.jl/stable/datasets/Mutagenesis/) for your convenience).

In [1]:
using MLDatasets, JsonGrinder, Flux, Mill, MLDataPattern, Statistics, ChainRulesCore

[ Info: Installing scipy via the Conda scipy package...
[ Info: Running `conda install -q -y scipy` in root environment
Solving environment: ...working... done

## Package Plan ##

  environment location: /home/runner/.julia/conda/3

  added / updated specs:
    - scipy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    libgfortran-ng-7.5.0       |      ha8ba4b0_17          22 KB
    libgfortran4-7.5.0         |      ha8ba4b0_17         995 KB
    scipy-1.7.1                |   py39h292c36d_2        16.9 MB
    ------------------------------------------------------------
                                           Total:        17.9 MB

The following NEW packages will be INSTALLED:

  libgfortran-ng     pkgs/main/linux-64::libgfortran-ng-7.5.0-ha8ba4b0_17
  libgfortran4       pkgs/main/linux-64::libgfortran4-7.5.0-ha8ba4b0_17
  scipy              pkgs/main/linux-64::scipy-1.7.1-py39h292c36

start by loading all samples

In [2]:
train_x, train_y = MLDatasets.Mutagenesis.traindata();
test_x, test_y = MLDatasets.Mutagenesis.testdata();
minibatchsize = 100
iterations = 5_000
neurons = 20 		# neurons per layer

20

 Create the schema and extractor from training data

In [3]:
sch = JsonGrinder.schema(train_x)
extractor = suggestextractor(sch)

Dict
  ├─── lumo: Categorical d = 99
  ├─── inda: Categorical d = 2
  ⋮
  └── atoms: Array of
               └── Dict
                     ⋮

 Convert samples to Mill structure and extract targets

In [4]:
train_data = extractor.(train_x)
test_data = extractor.(test_x)
labelnames = unique(train_y)

@show train_data[1]

train_data[1] = ProductNode


ProductNode 	# 1 obs, 104 bytes
  ├─── lumo: ArrayNode(99×1 OneHotArray with Bool elements) 	# 1 obs, 60 bytes
  ├─── inda: ArrayNode(2×1 OneHotArray with Bool elements) 	# 1 obs, 60 bytes
  ├─── logp: ArrayNode(63×1 OneHotArray with Bool elements) 	# 1 obs, 60 bytes
  ├─── ind1: ArrayNode(3×1 OneHotArray with Bool elements) 	# 1 obs, 60 bytes
  └── atoms: BagNode 	# 1 obs, 136 bytes
               └── ProductNode 	# 26 obs, 64 bytes
                     ⋮

# Create the model

In [5]:
model = reflectinmodel(sch, extractor,
	layer -> Dense(layer, neurons, relu),
	bag -> SegmentedMeanMax(bag),
	fsm = Dict("" => layer -> Dense(layer, length(labelnames))),
)

ProductModel ↦ ArrayModel(Dense(100, 2)) 	# 2 arrays, 202 params, 888 bytes
  ├─── lumo: ArrayModel(Dense(99, 20, relu)) 	# 2 arrays, 2_000 params, 7.891 KiB
  ├─── inda: ArrayModel(Dense(2, 20, relu)) 	# 2 arrays, 60 params, 320 bytes
  ├─── logp: ArrayModel(Dense(63, 20, relu)) 	# 2 arrays, 1_280 params, 5.078 KiB
  ├─── ind1: ArrayModel(Dense(3, 20, relu)) 	# 2 arrays, 80 params, 400 bytes
  └── atoms: BagModel ↦ [SegmentedMean(20); SegmentedMax(20)] ↦ ArrayModel(Dense(40, 20, relu)) 	# 4 arrays, 860 params, 3.516 KiB
               └── ProductModel ↦ ArrayModel(Dense(61, 20, relu)) 	# 2 arrays, 1_240 params, 4.922 KiB
                     ⋮

# Train the model
let's define loss and some helper functions

In [6]:
loss(x,y) = Flux.logitcrossentropy(inference(x), Flux.onehotbatch(y, labelnames))
inference(x::AbstractMillNode) = model(x).data
inference(x::AbstractVector{<:AbstractMillNode}) = inference(reduce(catobs, x))
accuracy(x,y) = mean(labelnames[Flux.onecold(inference(x))] .== y)
loss(xy::Tuple) = loss(xy...)
@non_differentiable Base.reduce(catobs, x::AbstractVector{<:AbstractMillNode})
cb = () -> begin
	train_acc = accuracy(train_data, train_y)
	test_acc = accuracy(test_data, test_y)
	println("accuracy: train = $train_acc, test = $test_acc")
end

#9 (generic function with 1 method)

create minibatches

In [7]:
minibatches = RandomBatches((train_data, train_y), size = minibatchsize, count = iterations)
Flux.Optimise.train!(loss, Flux.params(model), minibatches, ADAM(), cb = Flux.throttle(cb, 2))

accuracy: train = 0.4, test = 0.29545454545454547
accuracy: train = 0.82, test = 0.8863636363636364
accuracy: train = 0.85, test = 0.8863636363636364
accuracy: train = 0.86, test = 0.8863636363636364
accuracy: train = 0.85, test = 0.8863636363636364
accuracy: train = 0.85, test = 0.8863636363636364
accuracy: train = 0.86, test = 0.8863636363636364
accuracy: train = 0.85, test = 0.8863636363636364
accuracy: train = 0.86, test = 0.8863636363636364
accuracy: train = 0.86, test = 0.8863636363636364
accuracy: train = 0.85, test = 0.8863636363636364
accuracy: train = 0.86, test = 0.8863636363636364
accuracy: train = 0.87, test = 0.8863636363636364
accuracy: train = 0.87, test = 0.8863636363636364
accuracy: train = 0.87, test = 0.8863636363636364
accuracy: train = 0.87, test = 0.8863636363636364
accuracy: train = 0.88, test = 0.8863636363636364
accuracy: train = 0.9, test = 0.8863636363636364
accuracy: train = 0.9, test = 0.8863636363636364
accuracy: train = 0.91, test = 0.8863636363636364
ac

# Classify test set

In [8]:
probs = softmax(inference(test_data))
o = Flux.onecold(probs)
pred_classes = labelnames[o]
print(mean(pred_classes .== test_y))

0.8636363636363636

we see the accuracy is around 79% on test set

In [9]:
#predicted classes for test set
print(pred_classes)

[1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1]

gt classes for test set

In [10]:
print(test_y)

[1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1]

probabilities for test set

In [11]:
print(probs)

Float32[0.7966703 0.06769236 0.7126265 0.051600363 0.06066731 0.7419391 0.04184219 0.14291018 0.9724018 0.80198383 0.70539814 0.04861571 0.049208514 0.79428434 0.7124812 0.14294115 0.7840662 0.049588382 0.052255884 0.14241649 0.0067686387 0.96804017 0.7354195 0.990464 0.17703372 0.82597524 0.4959888 0.076788634 0.6863598 0.67669773 0.72284144 0.056862473 0.17945196 0.058966007 0.794795 0.7890754 0.905496 0.7967262 0.9926622 0.06749877 0.8608849 0.71532446 0.13802122 0.8161699; 0.20332964 0.93230766 0.28737348 0.9483996 0.93933266 0.25806087 0.9581578 0.8570898 0.027598204 0.19801615 0.2946019 0.95138425 0.9507915 0.20571566 0.28751886 0.8570588 0.2159338 0.9504117 0.9477441 0.8575836 0.9932314 0.031959876 0.2645805 0.00953606 0.8229663 0.1740247 0.5040112 0.9232114 0.3136402 0.32330227 0.27715862 0.9431376 0.82054806 0.94103396 0.20520507 0.21092467 0.09450401 0.2032737 0.007337767 0.93250126 0.13911508 0.2846755 0.8619788 0.18383014]

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*